In [ ]:
# Global Imports
import glob
import os
import warnings
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt

# import xagg as xa
import numpy as np
import pandas as pd
import rioxarray
import xarray as xr
from shapely import wkt
from shapely.geometry import mapping

warnings.filterwarnings("ignore", message=".*initial implementation of Parquet.*")

In [ ]:
region_file = "../data/raw/region/region.geojson"

In [ ]:
gdf = gpd.read_file(region_file, crs="EPSG:4326")

In [ ]:
# Clip CHIRPS to region

In [ ]:
chirps_dir = "../data/raw/chirps/*.tif"

In [ ]:
chirps_list = sorted(glob.glob(chirps_dir))

In [ ]:
chirps_time = xr.Variable(
    "time", [pd.to_datetime(f"{Path(_f).name[:-4]}") for _f in chirps_list]
)

In [ ]:
chirps_ds = xr.concat(
    [
        xr.open_rasterio(f).squeeze("band").drop("band").to_dataset(name="precip")
        for f in chirps_list
    ],
    dim=chirps_time,
)

In [ ]:
monthly_chirps = (
    chirps_ds.resample(time="1MS").mean(dim="time").rename({"y": "lat", "x": "lon"})
)

In [ ]:
monthly_chirps.to_netcdf("../data/processed/chirps.nc")

In [ ]:
# Convert Terra Data to netcdf

In [ ]:
terra_dir = "../data/raw/terra_climate/*.tif"

In [ ]:
terra_list = sorted(glob.glob(terra_dir))

In [ ]:
Path(terra_list[0]).name[:-4]

In [ ]:
terra_time = xr.Variable(
    "time", [pd.to_datetime(f"{Path(_f).name[:-4]}01") for _f in terra_list]
)

In [ ]:
terra_climate_bands = ["aet", "def", "pdsi", "pet", "pr", "srad", "ro", "soil", "swe"]

In [ ]:
terra_da = xr.concat(
    [xr.open_rasterio(f).rename({"y": "lat", "x": "lon"}) for f in terra_list],
    dim=terra_time,
)

In [ ]:
terra_da["band"] = terra_climate_bands

In [ ]:
terra_ds = terra_da.to_dataset(dim="band")

In [ ]:
fig, ax = plt.subplots(1, 2)
terra_ds.pdsi.isel(time=100).plot(ax=ax[1])
gdf.plot(ax=ax[1], edgecolor="red", color="none")
chirps_ds.precip.isel(time=100).plot(ax=ax[0])
gdf.plot(ax=ax[0], edgecolor="red", color="none")

In [ ]:
terra_ds.to_netcdf("../data/processed/terra.nc")

In [ ]:
terra_ds.aet.isel(time=10).plot()

In [ ]:
chirps_ds.precip.isel(time=10).plot()

In [ ]:
# Clip GRACE
# Loading GRACE dataset
grace = xr.open_dataset(
    "../data/raw/mascon/GRCTellus.JPL.200204_202206.GLO.RL06M.MSCNv02CRI.nc"
)["lwe_thickness"].to_dataset()

In [ ]:
# Calculate GRACE TWS Anomalies
# Clipping the grace ds to mississippi shapefile
grace = grace.assign({"lon": (((grace.lon + 180) % 360) - 180)}).sortby("lon")
grace["lwe_thickness"] = grace["lwe_thickness"].rio.write_crs("epsg:4326")
grace.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)

In [ ]:
grace_clipped = grace.rio.clip(
    gdf.geometry.apply(mapping), gdf.crs, all_touched=True, drop=True
)

In [ ]:
final_grace = grace_clipped.resample(time="1MS").mean(dim="time")

In [ ]:
final_grace.to_netcdf("../data/processed/grace.nc")